# Kew Coding Club Challenges - Using APIs
### 2019-01-30

## Challenge 1

Use the GBIF API to get the total number of occurrence records held in their database (this website lists all of the available endpoints for occurrences: https://www.gbif.org/developer/occurrence) 

In [1]:
import requests

url = "http://api.gbif.org/v1/occurrence/count"
response = requests.get(url)
f"There are {response.json():,} occurrence records in GBIF"

'There are 1,388,149,604 occurrence records in GBIF'

## Challenge 2

Use the GBIF API to get the total number of occurrence records for *Myrcia decortans*.

In [2]:
occurrence_url = "http://api.gbif.org/v1/occurrence/count"
backbone_url = "http://api.gbif.org/v1/species/match"
species = "Myrcia decortans"

response = requests.get(backbone_url, params={"name": species})
species_results = response.json()
species_key = species_results["usageKey"]

response = requests.get(occurrence_url, params={"taxonKey": species_key})
n_records = response.json()
f"There are {n_records:,} occurrence records for {species} in GBIF"

'There are 696 occurrence records for Myrcia decortans in GBIF'

## Challenge 3

Use the GBIF API to get all occurrence records for *Myrcia decortans* that have coordinates without any geospatial issues.

In [3]:
import pandas as pd

search_url = "http://api.gbif.org/v1/occurrence/search"
params = {
    "taxonKey": species_key,
    "hasCoordinate": "true",
    "hasGeospatialIssue": "false",
    "limit": 300,
    "offset": 0
}

response = requests.get(search_url, params=params)
parsed_response = response.json()

total_records = parsed_response["count"]
results = parsed_response["results"]

while len(results) < total_records:
    params["offset"] += len(results)
    response = requests.get(search_url, params=params)
    parsed_response = response.json()
    
    results.extend(parsed_response["results"])
    
occurrence_records = pd.DataFrame(results)

occurrence_records.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Columns: 117 entries, acceptedScientificName to year
dtypes: float64(9), int64(11), object(97)
memory usage: 429.7+ KB


In [4]:
occurrence_records.head()

,acceptedScientificName,acceptedTaxonKey,accessRights,basisOfRecord,bibliographicCitation,catalogNumber,class,classKey,collectionCode,collectionID,...,taxonRank,taxonomicStatus,type,typeStatus,typifiedName,verbatimElevation,verbatimEventDate,verbatimLocality,vernacularName,year
0,Myrcia decorticans DC.,3174041,NaN,HUMAN_OBSERVATION,NaN,KwCu9_69,Magnoliopsida,220,NTFP_Kw_Cu,NaN,...,SPECIES,ACCEPTED,NaN,NaN,NaN,NaN,NaN,NaN,saha,2017.0
1,Myrcia decorticans DC.,3174041,NaN,PRESERVED_SPECIMEN,NaN,80323,Magnoliopsida,220,IPA,NaN,...,SPECIES,ACCEPTED,Collection,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
2,Myrcia decorticans DC.,3174041,NaN,PRESERVED_SPECIMEN,NaN,63712,Magnoliopsida,220,JPB,NaN,...,SPECIES,ACCEPTED,Collection,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
3,Myrcia decorticans DC.,3174041,NaN,PRESERVED_SPECIMEN,NaN,64148,Magnoliopsida,220,MAC,NaN,...,SPECIES,ACCEPTED,Collection,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
4,Myrcia decorticans DC.,3174041,NaN,PRESERVED_SPECIMEN,NaN,RB01421744,Magnoliopsida,220,RB,NaN,...,SPECIES,ACCEPTED,Physical Object,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
